# Welcome to the Bird Probability Project

In [1]:
import pandas as pd
import numpy as np
import datetime 


In [2]:
# data = pd.read_csv('ebd_US-CO_202001_202312_smp_relMar-2024/ebd_US-CO_202001_202312_smp_relMar-2024.txt', sep = '\t')
# data = data[['COMMON NAME', 'COUNTY', 'COUNTY CODE', 'LATITUDE', 'LONGITUDE', 'OBSERVATION DATE', 'NUMBER OBSERVERS']]
# data.to_csv('bird_data.csv')

data = pd.read_csv('bird_data.csv')

data.head(5)

,Unnamed: 0,COMMON NAME,COUNTY,COUNTY CODE,LATITUDE,LONGITUDE,OBSERVATION DATE,NUMBER OBSERVERS
0,0,Accipiter sp.,Adams,US-CO-001,39.947339,-104.747209,2020-01-10,1.0
1,1,American Crow,Adams,US-CO-001,39.797274,-104.931060,2020-01-01,5.0
2,2,American Crow,Adams,US-CO-001,39.789034,-104.905840,2020-01-02,1.0
3,3,American Crow,Adams,US-CO-001,39.830492,-104.948991,2020-01-01,10.0
4,4,American Crow,Adams,US-CO-001,39.830492,-104.948991,2020-01-01,10.0


In [39]:
counties = data['COUNTY'].unique()
birds = data['COMMON NAME'].unique()

rawDate = datetime.datetime.now()

month = rawDate.month
day = rawDate.day


def getBirds(day, month, plusOrMinus):
    """
    returns a data frame of birds that have been observed from various years during the same day and month
    plus_or_minus so many days.
    
    """
    def distance(date, target):
        '''
        Helper function
        returns the number of days away the date is from the target
        '''
        givenMonth = int(date[5:7])
        targetMonth = int(target[5:7])
        givenDay = int(date[8:])
        targetDay = int(target[8:])
        
        total = abs(givenMonth - targetMonth)*30 #grab the difference in months
        
        if givenMonth > targetMonth:
            total = total - targetDay + givenDay
        elif givenMonth < targetMonth:
            total = total - givenDay + targetDay
        else: 
            total = abs(givenDay - targetDay)
        
        return total
        
     
    if month > 10:
        target = 'xxxx-{}-{}'.format(month,day) if day > 10 else 'xxxx-{}-0{}'.format(month,day)
    else:
        target = 'xxxx-0{}-{}'.format(month,day) if day > 10 else 'xxxx-0{}-0{}'.format(month,day)
        

    # This is not the best implementation, I am just lazy and didn't want to search for the right one (help if you can)
    mask = pd.Series([distance(date, target) <= plusOrMinus for date in data['OBSERVATION DATE']])
        
    return data[mask]


class County:
    def __init__(self, name, birds, birdCounts):
        self.name = name
        self.features = {bird: 0.0 for bird in birds}
        self.birdCounts = birdCounts #dictionary of bird name and count 
        
    def getFeatures(self):
        total = sum(list(self.birdCounts.values()))
        for bird in self.birdCounts:
            self.features[bird] = self.birdCounts[bird]/total
        
    
class User:
    def __init__(self, ratings, birds):
        self.features = {bird: 0.0 for bird in birds}
        self.ratings = ratings
        
    def getFeatures(self):
        total = sum(list(self.ratings.values()))
        for bird in self.ratings:
            self.features[bird] = self.ratings[bird]/total


def cosSim(a, b):           
    return np.dot(a, b)/(np.linalg.norm(a)*np.linalg.norm(b))
    

### Concern:
Features are going to be the proportion of that bird in the area. Should this be weighted by count? If so that could skew the data so cities with higher populations get better ratings, so probably not. 

In [7]:
seasonal_birds_within_20_days = getBirds(day,month, 10)

unique_birds_in_season = seasonal_birds_within_20_days['COMMON NAME'].unique()

groupByCounty = data.groupby(by = ["COUNTY"])['COMMON NAME'].value_counts()

groupByCounty


COUNTY  COMMON NAME               
Adams   Canada Goose                  19275
        European Starling             18137
        Red-winged Blackbird          17126
        Mallard                       17013
        Red-tailed Hawk               14750
                                      ...  
Yuma    Lesser/Greater Yellowlegs         1
        Loggerhead/Northern Shrike        1
        Fox Sparrow                       1
        Eurasian/American Wigeon          1
        wren sp.                          1
Name: count, Length: 22281, dtype: int64

In [33]:
countyFeatures = []

for county in counties:
    counts = {i: groupByCounty[county][i] for i in groupByCounty[county].index}
    countyFeatures.append(County(county, birds, counts))


for county in countyFeatures:
    county.getFeatures()

In [51]:
ratings = {'Bald Eagle': 5, "Red-winged Blackbird": 4, 'Red-tailed Hawk': 5}

user = User(ratings, birds)
user.getFeatures()

countyScore = []

for county in countyFeatures:
    countyScore.append((county.name, cosSim(list(county.features.values()), list(user.features.values()))))
    
countyScore = sorted(countyScore, key = lambda x: x[1])

print('The top county bases on your ratings are: ', [x[0] for x in countyScore[:5]])



The top county bases on your ratings are:  ['Clear Creek', 'San Juan', 'Summit', 'Gilpin', 'Pitkin']
